In [ ]:
#Import Libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import time

start_time = time.time()

In [ ]:
# Create path to Prepared data folder
path_prepared = r'C:\Users\danie\Desktop\career Foundry\31-03-2023 Instacart Basket Analysis\02 Data\Prepared data'

In [ ]:
# Create path to 'Visualizations' folder
path_visualizations = r'C:\Users\danie\Desktop\career Foundry\31-03-2023 Instacart Basket Analysis\04 Analysis\Visualizations'

In [ ]:
# Create path to Original data folder
path_original = r'C:\Users\danie\Desktop\career Foundry\31-03-2023 Instacart Basket Analysis\02 Data\Original data'

In [ ]:
# Import Instacart_data_set file
df_final = pd.read_pickle(os.path.join(path_prepared, 'instacart_data_set.pkl'))

In [ ]:
df_final['spender_flag'].value_counts()

In [ ]:
df_final.shape

In [ ]:
# Removing PII data to address security implications

df_final.drop(columns = ['first_name', 'surname'], inplace = True)

In [ ]:
#Verifying columns removal

df_final.columns

## Busiest Days of the Week

In [ ]:
# Create a bar chart 'orders_day_of_the_week'
busiest_days = df_final['orders_day_of_the_week'].value_counts().loc[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
plt.rcParams['figure.figsize'] = [10, 5]
plt.title('Busiest Days of the Week', fontweight = 'bold')
plt.xlabel('Days of the Week')
busiest_day = busiest_days.plot.bar(rot=0)

In [ ]:
#Create a histogram with 'order_hour_of_day' using '.plot.hist()' function
hist = df_final['order_hour_of_day'].plot.hist(bins = 100)
plt.title('Busiest Hours of the Day', fontweight = 'bold')
plt.xlabel('Hour of the day')

In [ ]:
#Defining df 
dfspend_hour = pd.crosstab(df_final['order_hour_of_day'], df_final['prices'].sum())

In [ ]:
dfspend_hour.shape

In [ ]:
dfspend_hour = pd.DataFrame(np.row_stack([dfspend_hour.columns, dfspend_hour.values]),
    columns=['revenue'])

In [ ]:
dfspend_hour.head()

In [ ]:
dfspend_hour = dfspend_hour.reset_index()

In [ ]:
dfspend_hour.head()

In [ ]:
dfspend_hour = dfspend_hour.rename(columns = {'index': 'hour'})

In [ ]:
dfspend_hour.drop(0, axis=0, inplace=True)


In [ ]:
dfspend_hour

In [ ]:
fig, ax = plt.subplots()
ind = np.arange(len(dfspend_hour['hour']))
 
ax.barh(ind, dfspend_hour['revenue'])
 
for i, v in enumerate(dfspend_hour['revenue']):
    ax.text(v, i, str(v))
plt.rcParams['figure.figsize'] = [15, 10]
plt.show()


In [ ]:
dfspend_hour.describe()

In [ ]:
summary_bar = dfspend_hour['revenue'].describe().plot.bar(title ='Revenue', rot=0)

for i in summary_bar.patches:
    plt.text(i.get_x() + 0.2, i.get_height()+ 200, \
             str(round((i.get_height()), 2)), fontsize=10, color='black', weight = 'bold')

In [ ]:
#Visual categorization Age groups.
revenue_pie = dfspend_hour['revenue'].plot.pie(title ='Age Groups', autopct=('%1.f%%'))

In [ ]:
revenue_bar = dfspend_hour['revenue'].plot.bar(title ='Revenue', color= (0, 0, 1, 0.3), rot=9)
plt.xticks(range(len(dfspend_hour)), dfspend_hour.index)
for i in revenue_bar.patches:
    plt.text(i.get_x() + 0.2, i.get_height()+ 200, \
             str(round((i.get_height()), 2)), fontsize=10, color='black')

# show the plot
plt.show()

In [ ]:
df_final.columns

In [ ]:
#Check Data consistency
df_final['region'].value_counts(dropna = False)

In [ ]:
#Create df_region_spend using .crosstab() to select the necessary data.
df_region_spend = pd.crosstab(df_final['region'], df_final['spender_flag'], dropna = False)

In [ ]:
df_region_spend

In [ ]:
df_region_spend.describe()

In [ ]:
#Creating 'exclusion_flag' based on customers order activity

result = []

for x in df_final['max_orders']:
    if x < 5:
        result.append('Low activity')
    else:
        result.append('High activity')

df_final['exclusion_flag'] = result

In [ ]:
df_final['exclusion_flag'].value_counts()

# Creating a subset of high order customer

In [ ]:
df_high_act_profiles = df_final.drop(df_final[df_final.exclusion_flag == 'Low activity'].index)

In [ ]:
df_high_act_profiles.shape

In [ ]:
#Export DataFrame with exclusion flag
df_high_act_profiles.to_pickle(os.path.join(path_prepared, 'instacart_data_set_with_exclusion.pkl'))

## Profiling Customer in Database

### Age profiling

In [ ]:
#Grouping customers by age using .quantile() as form of division

q_1 = df_final.age.quantile(0.25)
q_2 = df_final.age.quantile(0.50)
q_3 = df_final.age.quantile(0.75)

q_1, q_2, q_3 

In [ ]:
#Grouping customers usin 'for_in' and 'if else' functions
young = []

for x in df_final['age']:
    if x <= q_1:
        young.append('18-33')
    elif x > q_1 and x <= q_2:
        young.append('34-49')
    elif x > q_2 and x <= q_3:
        young.append('50-65')
    elif x > q_3:
        young.append('66+')
    else:
        young.append('Unkonwn')

df_final['age_range'] = young

df_final['age_range'].value_counts()

In [ ]:
#Visual categorization Age groups.
age_pie = df_final['age_range'].value_counts().plot.pie(title ='Age Groups', label = '', autopct=('%1.f%%'))

### Age-Region Crosstab for visualization

In [ ]:
#Defining df with age and region
age_region = pd.crosstab(df_final['region'], df_final['age_range'])

In [ ]:
barh_age_region = age_region.plot.barh(color = sns.color_palette('Oranges', 4))
plt.title('Age group population by region', fontweight = 'bold')
plt.xlabel('Frequency')

In [ ]:
#Age-region summary
age_region.describe()

### Import Department Data set

In [ ]:
# Import Department Data set
df_dept = pd.read_csv(os.path.join(path_prepared, 'departments_wrangled.csv'))

In [ ]:
df_dept.head()

In [ ]:
#Rename columns with a self-descriptive name
df_dept.rename(columns = {'Unnamed: 0' : 'department_id'}, inplace = True)

In [ ]:
df_dept.head()

### Merge Department Data set with the main Data set

In [ ]:
#Using .merge() funtion to merge data sets
df_final = df_final.merge(df_dept, on = 'department_id')

In [ ]:
df_final

### Price range and age range correlation

In [ ]:
#Use .crosstab() funtion 
age_spenditure = pd.crosstab(df_final['price_range_loc'], df_final['age_range'])

In [ ]:
age_spend = age_spenditure.plot.barh()
plt.title('Spenditure by Age group', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Price Range')

### Department and age range correlation

In [ ]:
age_department = pd.crosstab(df_final['department'], df_final['age_range'])

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
age_dept = age_department.plot.barh()
plt.title('Sales by Department, Age group', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Department',fontweight = 'bold')

### Orders day of the week and age range correlation

In [ ]:
age_day = pd.crosstab(df_final['orders_day_of_the_week'], df_final['age_range']).loc[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

In [ ]:
age_days = age_day.plot.barh()
plt.title('Orders day of the week and Age Range', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Day of the week',fontweight = 'bold')

## Family Size Profiling

## Grouping Customers based on Number of dependents

In [ ]:
#Using 'for in' funtion
parent = []

for y in df_final['number_of_kids']:
    
    if y ==  0:
        parent.append('Single adult')
    elif y == 1:
        parent.append('Reg family, 1 depnds')
    elif y == 2:
        parent.append('Reg family, 2 depnds')
    else:
        parent.append('Big family, 3+ depnds')

df_final['Family_size'] = parent

In [ ]:
#Count of customers based on family size
df_final['Family_size'].value_counts()

### Orders day of the week, age range, Family size comparison

In [ ]:
age_cross = pd.crosstab(df_final['orders_day_of_the_week'], [df_final['Family_size'], df_final['age_range']]).loc[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
age_size_day = age_cross.plot.barh(color = sns.color_palette('Greens', 16))
plt.title('Orders day of the week, age range, Family size', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Day of the week',fontweight = 'bold')

### Orders Hour of the Day-Age range-Family size comparison

In [ ]:
size_age_hour = pd.crosstab(df_final['order_hour_of_day'], [df_final['Family_size'], df_final['age_range']])

In [ ]:
plt.rcParams['figure.figsize'] = [20, 20]
age_size_hour = size_age_hour.plot.barh(color = sns.color_palette('Greens', 16))
plt.title('Orders Hour of the Day-Age range-Family size', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Hour of the Day',fontweight = 'bold')

### Sales comparison between Departments based on Family size 

In [ ]:
dept_size = pd.crosstab(df_final['department'], [df_final['Family_size']])

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
size_dept = dept_size.plot.barh(color = sns.color_palette('Reds', 16))
plt.title('Sales comparison between Departments based on Family size', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Department',fontweight = 'bold')

### Sales Comparison Age ranges, Family Sizes based on Departments

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
bars = pd.concat([age_department, dept_size]).describe().plot.bar(rot=0)
plt.title('Sales summary Departments, Family size', fontweight = 'bold')
plt.xlabel('Summary, MEAN, MAX,...')

***Summary chart Age ranged & Family Sizes Comparison based on Departments***

###  Sales Comparison between Regions based on Departments 

In [ ]:
dept_region = pd.crosstab(df_final['department'], [df_final['region']])

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
region_dept = dept_region.plot.barh()
plt.title('Sales within Departments by Region', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Department',fontweight = 'bold')


# Income Profiling

In [ ]:
#Summary 'Income'
df_final['income'].describe()


## Grouping Customers based on their income

In [ ]:
#Grouping Customers based on Number of dependents
#Using .quantile() funtions as reference for the division

low = df_final.income.quantile(0.25)
median = df_final.income.quantile(0.50)
high = df_final.income.quantile(0.75)
in_group = []

for i in df_final['income']:
    if i < low:
        in_group.append('Low income')
    elif i >= low and i < median:
        in_group.append('Low-avg income')
    elif i >= median and i < high:
        in_group.append('High-avg income')
    elif i >= high:
        in_group.append('High income')
    else:
        in_group.append('Unknown')
        

In [ ]:
#Creating income flag
df_final['income_group'] = in_group

In [ ]:
df_final['income_group'].value_counts()

## Customers income group per Region

In [ ]:
#Use .crosstab() function 
income_region = pd.crosstab(df_final['region'], df_final['income_group'])

In [ ]:
region_income = income_region.plot.bar(color = sns.color_palette('Greens', 7), rot = 0)
plt.rcParams['figure.figsize'] = [5, 5]
plt.title('Customers income group per Region', fontweight = 'bold')
plt.xlabel('Region')

### Sales Comparison between regions based on Price range

In [ ]:
region_prices = pd.crosstab(df_final['price_range_loc'], df_final['region'])

In [ ]:
price_region = region_prices.plot.bar(rot=0)
plt.rcParams['figure.figsize'] = [15, 5]
plt.title('Sales within Price range per region', fontweight = 'bold')
plt.xlabel('Price Range')

### Sales Comparison between Income groups based on Price range

In [ ]:
income_prices = pd.crosstab(df_final['income_group'], df_final['price_range_loc'])

In [ ]:
inc = income_prices.plot.barh(color = sns.color_palette('magma', 7))
plt.rcParams['figure.figsize'] = [15, 5]
plt.title('Income groups orders per Price range', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('')

***Mid-range-price products purshased has the highest sales in each customer group. The chart shows small purshases behabior in Low Income group, where they tend to buy Low_rage products vs other groups.***

In [ ]:
income_prices

### Correlation between income groups and Purshase frequency

In [ ]:
inc_activity = pd.crosstab(df_final['income_group'], df_final['frequency_flag']).loc[['Low income', 'Low-avg income', 'High-avg income', 'High income']]

In [ ]:
act_inc = inc_activity.plot.bar(rot = 0)
plt.rcParams['figure.figsize'] = [15, 5]
plt.title('Frequeny flag, Income groups', fontweight = 'bold')
plt.ylabel('')

In [ ]:
df_final.loc[(df_final['age'] <= 30) & (df_final['number_of_kids'] > 0), 'parent_group'] = 'Young parent'
df_final.loc[(df_final['age'] > 30) & (df_final['number_of_kids'] > 0), 'parent_group'] = 'Regular parent'
df_final.loc[(df_final['number_of_kids'] == 0), 'parent_group'] = 'Without dependents'

In [ ]:
df_final['parent_group'].value_counts()

### Sales Comparison between Genders based on Departments

In [ ]:
gender_spenditure = pd.crosstab(df_final['gender'], df_final['department'])

In [ ]:
gen_spend = gender_spenditure.plot.bar(rot=0)
plt.rcParams['figure.figsize'] = [5, 10]
plt.title('Department by Gender', fontweight = 'bold')
plt.xlabel('')
plt.ylabel('')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:
df_final.columns, "", df_final.shape

# Importing the visualizations

gen_spend.figure.savefig(os.path.join(path_visualizations, '4_10_bar_gen_spend.png'),dpi=300, bbox_inches = "tight")

act_inc.figure.savefig(os.path.join(path_visualizations, '4_10_bar_act_inc.png'),dpi=300, bbox_inches = "tight")

inc.figure.savefig(os.path.join(path_visualizations, '4_10_bar_inc_prices.png'),dpi=300, bbox_inches = "tight")

price_region.figure.savefig(os.path.join(path_visualizations, '4_10_bar_price_region.png'),dpi=300, bbox_inches = "tight")

region_income.figure.savefig(os.path.join(path_visualizations, '4_10_bar_region_income.png'),dpi=300, bbox_inches = "tight")

region_dept.figure.savefig(os.path.join(path_visualizations, '4_10_bar_region_dept.png'),dpi=300, bbox_inches = "tight")

bars.figure.savefig(os.path.join(path_visualizations, '4_10_bar_summary.png'),dpi=300, bbox_inches = "tight")

size_dept.figure.savefig(os.path.join(path_visualizations, '4_10_bar_family_size_dept.png'),dpi=300, bbox_inches = "tight")

age_size_hour.figure.savefig(os.path.join(path_visualizations, '4_10_bar_age_family_size_hour.png'),dpi=300, bbox_inches = "tight")

age_size_day.figure.savefig(os.path.join(path_visualizations, '4_10_bar_age_family_size_day.png'),dpi=300, bbox_inches = "tight")

age_days.figure.savefig(os.path.join(path_visualizations, '4_10_bar_age_days.png'),dpi=300, bbox_inches = "tight")

age_dept.figure.savefig(os.path.join(path_visualizations, '4_10_bar_age_dept.png'),dpi=300, bbox_inches = "tight")

age_spend.figure.savefig(os.path.join(path_visualizations, '4_10_bar_age_price_range.png'),dpi=300, bbox_inches = "tight")

barh_age_region.figure.savefig(os.path.join(path_visualizations, '4_10_bar_age_region.png'),dpi=300, bbox_inches = "tight")

age_pie.figure.savefig(os.path.join(path_visualizations, '4_10_age_pie.png'),dpi=300, bbox_inches = "tight")

In [ ]:
timer = (time.time() - start_time)/60
print(timer)